<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_hugectr_hps-hps-tensorflow-triton-deployment/nvidia_logo.png" style="width: 90px; float: right;">

# HPS TensorRT Plugin Demo for PyTorch Trained Model

## Overview

This notebook demonstrates how to build and deploy the HPS-integrated TensorRT engine for the model trained with PyTorch.

For more details about HPS, please refer to [HugeCTR Hierarchical Parameter Server (HPS)](https://nvidia-merlin.github.io/HugeCTR/master/hierarchical_parameter_server/index.html).

## Installation

### Use NGC

The HPS TensorRT plugin is preinstalled in the 23.01 and later [Merlin PyTorch Container](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-pytorch): `nvcr.io/nvidia/merlin/merlin-pytorch:23.01`.

You can check the existence of the required libraries by running the following Python code after launching this container.

In [1]:
import ctypes
plugin_lib_name = "/usr/local/hps_trt/lib/libhps_plugin.so"
plugin_handle = ctypes.CDLL(plugin_lib_name, mode=ctypes.RTLD_GLOBAL)

## Configurations

First of all we specify the required configurations, e.g., the arguments needed for generating the dataset, the model parameters and the paths to save the model. We will use DLRM model which has one embedding table, bottom MLP layers, interaction layer and top MLP layers. Please note that the input to the embedding layer will be a dense key tensor of int32.

In [2]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
import struct

args = dict()

args["gpu_num"] = 1                               # the number of available GPUs
args["iter_num"] = 50                             # the number of training iteration
args["slot_num"] = 26                             # the number of feature fields in this embedding layer
args["embed_vec_size"] = 128                      # the dimension of embedding vectors
args["dense_dim"] = 13                            # the dimension of dense features
args["global_batch_size"] = 1024                  # the globally batchsize for all GPUs
args["max_vocabulary_size"] = 260000
args["vocabulary_range_per_slot"] = [[i*10000, (i+1)*10000] for i in range(26)]
args["combiner"] = "mean"

args["ps_config_file"] = "dlrm_pytorch.json"
args["embedding_table_path"] = "dlrm_pytorch_sparse.model"
args["onnx_path"] = "dlrm_pytorch.onnx"
args["modified_onnx_path"] = "dlrm_pytorch_with_hps.onnx"
args["np_key_type"] = np.int32
args["np_vector_type"] = np.float32

os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(map(str, range(args["gpu_num"])))

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def generate_random_samples(num_samples, vocabulary_range_per_slot, dense_dim, key_dtype = args["np_key_type"]):
    keys = list()
    for vocab_range in vocabulary_range_per_slot:
        keys_per_slot = np.random.randint(low=vocab_range[0], high=vocab_range[1], size=(num_samples, 1), dtype=key_dtype)
        keys.append(keys_per_slot)
    keys = np.concatenate(np.array(keys), axis = 1)
    numerical_features = np.random.random((num_samples, dense_dim)).astype(np.float32)
    labels = np.random.randint(low=0, high=2, size=(num_samples, 1))
    return keys, numerical_features, labels

## Train with PyTorch

We define the model graph for training with native PyTorch layers, i.e., `torch.nn.Embedding`, `torch.nn.Linear` and so on. We can then train the model and extract the trained weights of the embedding table.

In [4]:
import torch

class MLP(torch.nn.Module):
    def __init__(self,
                arch,
                name,
                out_activation,
                **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.mlp = torch.nn.Sequential()
        
        for idx in range(1, len(arch)-1):
            self.mlp.add_module(name + "_linear_layer_%d" % idx, torch.nn.Linear(arch[idx-1], arch[idx]))
            self.mlp.add_module(name + "_relu_layer_%d" % idx, torch.nn.ReLU(inplace=True))
            
        idx = len(arch) - 1
        if out_activation == "relu":
            self.mlp.add_module(name + "_linear_layer_%d" % idx, torch.nn.Linear(arch[idx-1], arch[idx]))
            self.mlp.add_module(name + "_relu_layer_%d" % idx, torch.nn.ReLU(inplace=True))
        elif out_activation == "sigmoid":
            self.mlp.add_module(name + "_linear_layer_%d" % idx, torch.nn.Linear(arch[idx-1], arch[idx]))
            self.mlp.add_module(name + "_relu_layer_%d" % idx, torch.nn.Sigmoid())

    def forward(self, x):
        y = self.mlp(x)
        return y

    
class SecondOrderFeatureInteraction(torch.nn.Module):
    def __init__(self):
        super(SecondOrderFeatureInteraction, self).__init__()

    def forward(self, inputs, num_feas):
        dot_products = torch.reshape(torch.matmul(inputs, torch.transpose(inputs, 1, 2)), (-1, num_feas * num_feas))
        indices = torch.tensor([i * num_feas + j for j in range(1, num_feas) for i in range(j)])
        flat_interactions = torch.index_select(dot_products, 1, indices)
        return flat_interactions    

class DLRM(torch.nn.Module):
    def __init__(self,
                 init_tensors,
                 embed_vec_size,
                 slot_num,
                 dense_dim,
                 arch_bot,
                 arch_top,
                 **kwargs):
        
        super(DLRM, self).__init__()
        self.embedding = torch.nn.Embedding.from_pretrained(init_tensors, freeze=False)
        
        self.embed_vec_size = embed_vec_size
        self.slot_num = slot_num
        self.dense_dim = dense_dim
        self.arch_bot = arch_bot
        self.arch_top = arch_top

        self.bot_mlp = MLP([self.dense_dim] + arch_bot, name = "bottom", out_activation='relu')
        self.interaction_layer = SecondOrderFeatureInteraction()
        self.interaction_out_dim = self.slot_num * (self.slot_num+1) // 2
        self.top_mlp = MLP([self.interaction_out_dim + self.arch_bot[-1]] + arch_top, name = "top", out_activation='sigmoid')
    
    def forward(self, inputs):
        categorical_features = inputs[0]
        numerical_features = inputs[1]
        
        embedding_vector = self.embedding(categorical_features)
        dense_x = self.bot_mlp(numerical_features)
        
        concat_features = torch.concat([embedding_vector, torch.reshape(dense_x, (-1, 1, self.arch_bot[-1]))], 1)
        
        Z = self.interaction_layer(concat_features, self.slot_num+1)
        z = torch.concat([dense_x, Z], 1)
        logit = self.top_mlp(z)
        return logit

In [5]:
def train(args):
    init_tensors = torch.Tensor(np.ones(shape=[args["max_vocabulary_size"], args["embed_vec_size"]], dtype=args["np_vector_type"]))
    
    model = DLRM(init_tensors, args["embed_vec_size"], args["slot_num"], args["dense_dim"],
                arch_bot = [512, 256, args["embed_vec_size"]],
                arch_top = [1024, 1024, 512, 256, 1])

    print(model)

    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    
    keys, numerical_features, labels = generate_random_samples(args["global_batch_size"]  * args["iter_num"], args["vocabulary_range_per_slot"], args["dense_dim"], args["np_key_type"])
    x0_iterator = iter(DataLoader(torch.from_numpy(keys), batch_size=args["global_batch_size"], shuffle=True, num_workers=0, pin_memory=False, drop_last=False))
    x1_iterator = iter(DataLoader(torch.from_numpy(numerical_features), batch_size=args["global_batch_size"], shuffle=True, num_workers=0, pin_memory=False, drop_last=False))
    y_iterator = iter(DataLoader(torch.from_numpy(labels).float(), batch_size=args["global_batch_size"], shuffle=True, num_workers=0, pin_memory=False, drop_last=False))
    
    
    for i in range(args["iter_num"]):
        inputs = [next(x0_iterator), next(x1_iterator)]
        labels = next(y_iterator)
        preds = model(inputs)
        loss = criterion(preds.squeeze(), labels.squeeze())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print("-"*20, "Step {}, loss: {}".format(i, loss),  "-"*20)
    return model

In [6]:
trained_model = train(args)
embedding_weights = trained_model.state_dict()["embedding.weight"]
print(embedding_weights)

DLRM(
  (embedding): Embedding(260000, 128)
  (bot_mlp): MLP(
    (mlp): Sequential(
      (bottom_linear_layer_1): Linear(in_features=13, out_features=512, bias=True)
      (bottom_relu_layer_1): ReLU(inplace=True)
      (bottom_linear_layer_2): Linear(in_features=512, out_features=256, bias=True)
      (bottom_relu_layer_2): ReLU(inplace=True)
      (bottom_linear_layer_3): Linear(in_features=256, out_features=128, bias=True)
      (bottom_relu_layer_3): ReLU(inplace=True)
    )
  )
  (interaction_layer): SecondOrderFeatureInteraction()
  (top_mlp): MLP(
    (mlp): Sequential(
      (top_linear_layer_1): Linear(in_features=479, out_features=1024, bias=True)
      (top_relu_layer_1): ReLU(inplace=True)
      (top_linear_layer_2): Linear(in_features=1024, out_features=1024, bias=True)
      (top_relu_layer_2): ReLU(inplace=True)
      (top_linear_layer_3): Linear(in_features=1024, out_features=512, bias=True)
      (top_relu_layer_3): ReLU(inplace=True)
      (top_linear_layer_4): Line

## Build the HPS-integrated TensorRT engine
In order to use HPS in the inference stage, we need to convert the embedding weights to the formats required by HPS first and create JSON configuration file for HPS.

Then we convert the PyTorch model to ONNX, and employ the ONNX GraphSurgoen tool to replace the native PyTorch embedding lookup layer with the placeholder of HPS TensorRT plugin layer.

After that, we can build the TensorRT engine, which is comprised of the HPS TensorRT plugin layer and the dense network.

### Step1: Prepare sparse model and JSON configuration file for HPS

Please note that the storage format in the `dlrm_pytorch_sparse.model/key` file is int64, while the HPS TensorRT plugin currently only support int32 when loading the keys into memory. There is no overflow since the key value range is 0~260000.

In [7]:
def convert_to_sparse_model(embeddings_weights, embedding_table_path, embedding_vec_size):
    os.system("mkdir -p {}".format(embedding_table_path))
    with open("{}/key".format(embedding_table_path), 'wb') as key_file, \
        open("{}/emb_vector".format(embedding_table_path), 'wb') as vec_file:
      for key in range(embeddings_weights.shape[0]):
        vec = embeddings_weights[key]
        key_struct = struct.pack('q', key)
        vec_struct = struct.pack(str(embedding_vec_size) + "f", *vec)
        key_file.write(key_struct)
        vec_file.write(vec_struct)

In [8]:
convert_to_sparse_model(embedding_weights.numpy(), args["embedding_table_path"], args["embed_vec_size"])

In [9]:
%%writefile dlrm_pytorch.json
{
    "supportlonglong": false,
    "models": [{
        "model": "dlrm",
        "sparse_files": ["dlrm_pytorch_sparse.model"],
        "num_of_worker_buffer_in_pool": 3,
        "embedding_table_names":["sparse_embedding0"],
        "embedding_vecsize_per_table": [128],
        "maxnum_catfeature_query_per_table_per_sample": [26],
        "default_value_for_each_table": [1.0],
        "deployed_device_list": [0],
        "max_batch_size": 1024,
        "cache_refresh_percentage_per_iteration": 0.2,
        "hit_rate_threshold": 1.0,
        "gpucacheper": 1.0,
        "gpucache": true
        }
    ]
}

Writing dlrm_pytorch.json


### Step2: Convert to ONNX and do ONNX graph surgery

In [10]:
dummy_keys = torch.randint(0, args["max_vocabulary_size"], (args["global_batch_size"], args["slot_num"]), dtype=torch.int32)
dummy_numerical_features = torch.randn(args["global_batch_size"], args["dense_dim"])
torch.onnx.export(trained_model, 
                  [dummy_keys, dummy_numerical_features],
                  args["onnx_path"], 
                  verbose = True, 
                  input_names = ["keys", "numerical_features"], 
                  output_names = ["output"], 
                  dynamic_axes = {'keys' : {0 : 'batch_size'}, 'numerical_features' : {0 : 'batch_size'}}
                 )

/tmp/ipykernel_52545/1281679600.py:35: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  indices = torch.tensor([i * num_feas + j for j in range(1, num_feas) for i in range(j)])


Exported graph: graph(%keys : Int(*, 26, strides=[26, 1], requires_grad=0, device=cpu),
      %numerical_features : Float(*, 13, strides=[13, 1], requires_grad=0, device=cpu),
      %embedding.weight : Float(260000, 128, strides=[128, 1], requires_grad=1, device=cpu),
      %bot_mlp.mlp.bottom_linear_layer_1.weight : Float(512, 13, strides=[13, 1], requires_grad=1, device=cpu),
      %bot_mlp.mlp.bottom_linear_layer_1.bias : Float(512, strides=[1], requires_grad=1, device=cpu),
      %bot_mlp.mlp.bottom_linear_layer_2.weight : Float(256, 512, strides=[512, 1], requires_grad=1, device=cpu),
      %bot_mlp.mlp.bottom_linear_layer_2.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %bot_mlp.mlp.bottom_linear_layer_3.weight : Float(128, 256, strides=[256, 1], requires_grad=1, device=cpu),
      %bot_mlp.mlp.bottom_linear_layer_3.bias : Float(128, strides=[1], requires_grad=1, device=cpu),
      %top_mlp.mlp.top_linear_layer_1.weight : Float(1024, 479, strides=[479, 1], req

In [11]:
# ONNX graph surgery to insert HPS the TensorRT plugin placeholder
import onnx_graphsurgeon as gs
from onnx import  shape_inference
import numpy as np
import onnx

graph = gs.import_onnx(onnx.load("dlrm_pytorch.onnx"))
saved = []

for node in graph.nodes:
    if node.name == "/embedding/Gather":
        categorical_features = gs.Variable(name="categorical_features", dtype=np.int32, shape=("unknown", 26))
        hps_node = gs.Node(op="HPS_TRT", attrs={"ps_config_file": "dlrm_pytorch.json\0", "model_name": "dlrm\0", "table_id": 0, "emb_vec_size": 128}, 
                           inputs=[categorical_features], outputs=[node.outputs[0]])
        graph.nodes.append(hps_node)
        saved.append(categorical_features)
        node.outputs.clear()
for i in graph.inputs:
    if i.name == "numerical_features":
        saved.append(i)
graph.inputs = saved

graph.cleanup().toposort()
onnx.save(gs.export_onnx(graph), "dlrm_pytorch_with_hps.onnx")

### Step3: Build the TensorRT engine

In [12]:
# build the TensorRT engine based on dlrm_pytorch_with_hps.onnx
import tensorrt as trt
import ctypes

plugin_lib_name = "/usr/local/hps_trt/lib/libhps_plugin.so"
handle = ctypes.CDLL(plugin_lib_name, mode=ctypes.RTLD_GLOBAL)

TRT_LOGGER = trt.Logger(trt.Logger.INFO)
EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

def build_engine_from_onnx(onnx_model_path):
    with trt.Builder(TRT_LOGGER) as builder, builder.create_network(EXPLICIT_BATCH) as network, trt.OnnxParser(network, TRT_LOGGER) as parser, builder.create_builder_config() as builder_config:        
        model = open(onnx_model_path, 'rb')
        parser.parse(model.read())

        profile = builder.create_optimization_profile()        
        profile.set_shape("categorical_features", (1, 26), (1024, 26), (1024, 26))    
        profile.set_shape("numerical_features", (1, 13), (1024, 13), (1024, 13))
        builder_config.add_optimization_profile(profile)
        engine = builder.build_serialized_network(network, builder_config)
        return engine

serialized_engine = build_engine_from_onnx("dlrm_pytorch_with_hps.onnx")
with open("dlrm_pytorch_with_hps.trt", "wb") as fout:
    fout.write(serialized_engine)
print("Succesfully build the TensorRT engine")

[01/03/2023-07:25:18] [TRT] [I] [MemUsageChange] Init CUDA: CPU +268, GPU +0, now: CPU 1035, GPU 497 (MiB)
[01/03/2023-07:25:20] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +170, GPU +46, now: CPU 1259, GPU 543 (MiB)
[01/03/2023-07:25:20] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars
[01/03/2023-07:25:20] [TRT] [W] onnx2trt_utils.cpp:377: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[01/03/2023-07:25:20] [TRT] [I] No importer registered for op: HPS_TRT. Attempting to import as plugin.
[01/03/2023-07:25:20] [TRT] [I] Searching for plugin: HPS_TRT, plugin_version: 1, plugin_namespace: 
=====================================================HPS Parse====================================================
[HCTR][07:25:20.652][INFO][

## Deploy HPS-integrated TensorRT engine on Triton

In order to deploy the TensorRT engine with the Triton TensorRT backend, we need to create the model repository and define the `config.pbtxt` first.

In [13]:
!mkdir -p model_repo/dlrm_pytorch_with_hps/1
!mv dlrm_pytorch_with_hps.trt model_repo/dlrm_pytorch_with_hps/1

In [14]:
%%writefile model_repo/dlrm_pytorch_with_hps/config.pbtxt

platform: "tensorrt_plan"
default_model_filename: "dlrm_pytorch_with_hps.trt"
backend: "tensorrt"
max_batch_size: 0
input [
  {
    name: "categorical_features"
    data_type: TYPE_INT32
    dims: [-1,26]
  },
  {
    name: "numerical_features"
    data_type: TYPE_FP32
    dims: [-1,13]
  }
]
output [
  {
      name: "output"
      data_type: TYPE_FP32
      dims: [-1,1]
  }
]
instance_group [
  {
    count: 1
    kind: KIND_GPU
    gpus:[0]

  }
]


Writing model_repo/dlrm_pytorch_with_hps/config.pbtxt


In [15]:
!tree model_repo/dlrm_pytorch_with_hps

model_repo/dlrm_pytorch_with_hps
├── 1
│   └── dlrm_pytorch_with_hps.trt
└── config.pbtxt

1 directory, 2 files


We can then launch the Triton inference server using the TensorRT backend. Please note that `LD_PRELOAD` is utilized to load the custom TensorRT plugin (i.e., HPS TensorRT plugin) into Triton.

Note: `Since Background processes not supported by Jupyter, please launch the Triton Server according to the following command independently in the background`.

> **LD_PRELOAD=/usr/local/hps_trt/lib/libhps_plugin.so tritonserver --model-repository=/hugectr/hps_trt/notebooks/model_repo/ --load-model=dlrm_pytorch_with_hps --model-control-mode=explicit**

If you successfully started tritonserver, you should see a log similar to following:


```bash
+----------+--------------------------------+--------------------------------+
| Backend  | Path                           | Config                         |
+----------+--------------------------------+--------------------------------+
| tensorrt | /opt/tritonserver/backends/ten | {"cmdline":{"auto-complete-con |
|          | sorrt/libtriton_tensorrt.so    | fig":"true","min-compute-capab |
|          |                                | ility":"6.000000","backend-dir |
|          |                                | ectory":"/opt/tritonserver/bac |
|          |                                | kends","default-max-batch-size |
|          |                                | ":"4"}}                        |
|          |                                |                                |
+----------+--------------------------------+--------------------------------+


+-----------------------+---------+--------+
| Model                 | Version | Status |
+-----------------------+---------+--------+
| dlrm_pytorch_with_hps | 1       | READY  |
+-----------------------+---------+--------+
```

We can then send the requests to the Triton inference server using the HTTP client.

In [16]:
import os
import shutil
import numpy as np
import tritonclient.http as httpclient
from tritonclient.utils import *

BATCH_SIZE = 1024

categorical_feature = np.random.randint(0,260000,size=(BATCH_SIZE,26)).astype(np.int32)
numerical_feature = np.random.random((BATCH_SIZE, 13)).astype(np.float32)

inputs = [
    httpclient.InferInput("categorical_features", 
                          categorical_feature.shape,
                          np_to_triton_dtype(np.int32)),
    httpclient.InferInput("numerical_features", 
                          numerical_feature.shape,
                          np_to_triton_dtype(np.float32)),                          
]
inputs[0].set_data_from_numpy(categorical_feature)
inputs[1].set_data_from_numpy(numerical_feature)


outputs = [
    httpclient.InferRequestedOutput("output")
]

model_name = "dlrm_pytorch_with_hps"

with httpclient.InferenceServerClient("localhost:8000") as client:
    response = client.infer(model_name,
                            inputs,
                            outputs=outputs)
    result = response.get_response()
    
    print("Prediction result is \n{}".format(response.as_numpy("output")))
    print("Response details:\n{}".format(result))


Prediction result is 
[[0.5128022 ]
 [0.51312006]
 [0.51246136]
 ...
 [0.5129204 ]
 [0.51302147]
 [0.513144  ]]
Response details:
{'model_name': 'dlrm_pytorch_with_hps', 'model_version': '1', 'outputs': [{'name': 'output', 'datatype': 'FP32', 'shape': [1024, 1], 'parameters': {'binary_data_size': 4096}}]}
